## Doing K nearest neighbors algorithm

This code will apply the K Nearest Neighbors algorithm (KNN) to the IMDB Dataset based on vectors computed using TF-IDF
We will use a training set of size 20000 for each class (i.e. 20k positive and 20k negative reviews)


In [1]:
import pickle
import numpy as np
import pandas as pd
import re
import matplotlib.pyplot as plt
from collections import Counter
from nltk.corpus import stopwords
from tqdm import tqdm
from sklearn.decomposition import PCA
import plotly.express as px


## Un comment this lines the first time you run the code
#import nltk
#nltk.download('stopwords')
#nltk.download('punkt')


In [2]:
with open('./normalized_TF_IDF_matrix.pckl', 'rb') as f:
    normalized_TF_IDF_matrix = pickle.load(f)

In [3]:
data = pd.read_csv("./IMDB_Dataset.csv")

In [4]:
data

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
...,...,...
49995,I thought this movie did a down right good job...,positive
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative


In [5]:
# Create the training set and test set
positive_Ix = list(data[data['sentiment']=="positive"].index)
negative_Ix = list(data[data['sentiment']=="negative"].index)

train_Ix = np.concatenate([positive_Ix[:20000], negative_Ix[:20000]])
test_Ix = np.concatenate([positive_Ix[20000:], negative_Ix[20000:]])


In [6]:
distances_test_set = np.dot(normalized_TF_IDF_matrix[train_Ix, :], normalized_TF_IDF_matrix[test_Ix, :].T)

In [ ]:
with open('./distances_matrix.pckl', 'wb') as f:
    pickle.dump(distances_test_set, f)

In [7]:
distances_test_set

array([[ 0.01625964,  0.10389918,  0.06022358, ...,  0.00192716,
         0.03192133,  0.03816214],
       [-0.01300093, -0.00939264,  0.05319725, ...,  0.00675629,
         0.01108029,  0.0352032 ],
       [-0.01202382, -0.00494609,  0.03482943, ...,  0.02332947,
         0.0025546 ,  0.02288932],
       ...,
       [ 0.06619464,  0.00858151,  0.00755849, ..., -0.01252939,
         0.02729918,  0.02095895],
       [ 0.01740645,  0.01111872, -0.01623552, ...,  0.02370896,
        -0.01482924,  0.02864754],
       [-0.00512813,  0.00626675, -0.00103581, ...,  0.01528623,
         0.00465495,  0.0256219 ]], shape=(40000, 10000))